# Практическое задание

**Настройка общих параметров модели через конфигурационный файл default.yaml**

**Установите в общем конфигурационном файле default.yaml параметры, рекомендуемые преподавателями для запуска обучения модели: epochs 50, batch 16, imgsz 640.**

**Загрузите отредактированный файл default.yaml на Платформу.**

In [ ]:
# Пути к папкам
images_path = os.path.join("images", "train")
labels_path = os.path.join("labels", "train")
output_path = "aug_data"

# Параметр: во сколько раз увеличить выборку
augmentation_factor = 2  # Укажите нужное количество

# Создаем папку для сохранения результатов, если она не существует
os.makedirs(output_path, exist_ok=True)

# Определяем список аугментаций
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.ShiftScaleRotate(
        shift_limit=0.1, 
        scale_limit=0.2, 
        rotate_limit=15, 
        border_mode=0, 
        p=0.5
    ),
    A.GaussNoise(p=0.2)
], bbox_params=A.BboxParams(
    format='yolo',
    label_fields=['class_labels'],
    min_area=0.0,
    min_visibility=0.0,
    check_each_transform=False  # Отключаем проверку после каждого преобразования
))

# Функция для чтения разметки из файла
def read_labels(label_file):
    labels = []
    with open(label_file, 'r') as f:
        for line in f:
            data = line.strip().split()
            class_id = int(data[0])
            x_center, y_center, width, height = map(float, data[1:])
            labels.append((class_id, x_center, y_center, width, height))
    return labels

# Функция для сохранения разметки в файл
def save_labels(label_file, bboxes):
    with open(label_file, 'w') as f:
        for class_id, bbox in bboxes:
            x_center, y_center, w, h = bbox
            f.write(f"{int(class_id)} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")

# Функция для коррекции bounding boxes
def clip_bbox(bbox):
    """
    Обрезка координат bounding box до диапазона [0.0, 1.0].
    Если после обрезки ширина или высота становятся <= 0, возвращаем None.
    """
    x_center, y_center, w, h = bbox

    # Вычисляем x_min, y_min, x_max, y_max
    x_min = x_center - w / 2
    y_min = y_center - h / 2
    x_max = x_center + w / 2
    y_max = y_center + h / 2

    # Обрезаем до диапазона [0.0, 1.0]
    x_min = max(0.0, x_min)
    y_min = max(0.0, y_min)
    x_max = min(1.0, x_max)
    y_max = min(1.0, y_max)

    # Пересчитываем ширину и высоту
    w = x_max - x_min
    h = y_max - y_min

    # Проверяем, что ширина и высота больше 0
    if w <= 0 or h <= 0:
        return None

    # Пересчитываем центр
    x_center = (x_min + x_max) / 2
    y_center = (y_min + y_max) / 2

    return x_center, y_center, w, h

# Функция для аугментации изображения и разметки
def augment_image(image, bboxes):
    # Подготовка bounding boxes для Albumentations
    bbox_list = []
    class_labels = []
    for b in bboxes:
        class_id, x_center, y_center, w, h = b
        clipped_bbox = clip_bbox((x_center, y_center, w, h))
        if clipped_bbox is not None:
            bbox_list.append(clipped_bbox)
            class_labels.append(class_id)
    if not bbox_list:
        # Если нет валидных bounding boxes, возвращаем исходное изображение
        return image, []

    augmented = transform(image=image, bboxes=bbox_list, class_labels=class_labels)

    # Корректируем bounding boxes после аугментации
    corrected_bboxes = []
    for class_id, bbox in zip(augmented['class_labels'], augmented['bboxes']):
        clipped_bbox = clip_bbox(bbox)
        if clipped_bbox is not None:
            corrected_bboxes.append((class_id, clipped_bbox))

    return augmented['image'], corrected_bboxes

# Основной цикл для обработки всех изображений и файлов разметки
for filename in os.listdir(images_path):
    if filename.endswith(".png") or filename.endswith(".jpg"):
        image_path = os.path.join(images_path, filename)
        label_path = os.path.join(labels_path, filename.rsplit('.', 1)[0] + ".txt")

        # Проверяем наличие файла разметки
        if os.path.exists(label_path):
            # Загружаем изображение
            image = cv2.imread(image_path)
            height, width, _ = image.shape

            # Загружаем разметку
            bboxes = read_labels(label_path)

            # Аугментация несколько раз для увеличения выборки
            for i in range(augmentation_factor):
                # Аугментация изображения и разметки
                aug_image, aug_bboxes = augment_image(image, bboxes)

                if not aug_bboxes:
                    # Если после аугментации нет валидных bounding boxes, пропускаем
                    continue

                # Сохранение аугментированного изображения с уникальным именем
                output_image_path = os.path.join(output_path, f"{filename.rsplit('.', 1)[0]}_aug_{i}.png")
                cv2.imwrite(output_image_path, aug_image)

                # Сохранение аугментированной разметки
                output_label_path = os.path.join(output_path, f"{filename.rsplit('.', 1)[0]}_aug_{i}.txt")
                save_labels(output_label_path, aug_bboxes)

                print(f"Обработано и сохранено: {output_image_path}")

**На выходе получим увеличение количества изображений тренировочной выборки до 180.**
**Полный объем выборки - 204 изображения**

In [2]:
def convert2cv(data, img_x, img_y):
    params = data.split()
    xc = int(float(params[1]) * img_x)
    yc = int(float(params[2]) * img_y)
    width = int(float(params[3]) * img_x)
    heigth = int(float(params[4]) * img_y)
    return xc, yc, width, heigth

In [3]:
def check_bb(images, labels, name):
    img = cv2.imread(os.path.join(images, name + '.jpg'))
    img_y, img_x = img.shape[0], img.shape[1]
    with open(os.path.join(labels, name + '.txt'), 'r') as l:
        bb = []
        for line in l:
            x, y, w, h = convert2cv(line, img_x, img_y)
            x0, y0 = x - w // 2, y + h // 2
            xw, yh = x + w // 2, y - h // 2
            bb.append(((x0, y0), (xw, yh)))
    for rect in bb:
        cv2.rectangle(img, rect[0], rect[1], (0,255,0), 3)
    cv2.imshow('Bounding boxes', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [6]:
pth = os.curdir
images = os.path.join('frames', 'Аугментация', 'images')
labels = os.path.join('frames', 'Аугментация', 'labels')
files = os.listdir(os.path.join(pth, images))
step = int(1 / 0.2) # 20% изображений
for i in range(0, len(files), step):
    check_bb(images, labels, files[i].split('.')[0])

**Разметка аугментированной выборки соответсвует необходимым условиям**

## Сруктура проекта 

**Для проекта используется следующая структура**

### Файл настроек используется для задания конфигураций обучений, проверки и инференса модели

#### Содержимое файле **default.yaml** - файл настроек:
**Основные параметры**:
* Задача обучения - детекция
* Модель - предобученная, yolo8n
* Обучение на 20 эпохах
* Время эпох переопределения эпох
* Размер батча - 16
* Размер изображения 640х640
* Заданы гиперпараметры:
  * lr0=lrf=0,01 (lerning rate не меняется в процессе обучения);
  * momentum = 0,937;
  * оптимизатор весов = 0.0005 и др.

### Запуск обучения с использованием конфигурационного файла

In [4]:
model_npt = YOLO("yolov8n.pt")
model_npt.train(data='data.yaml', cfg='default.yaml')